In [1]:
# Run this command to capture iPhone output
# !nc -ul 11111 > livelink.udp 2>&1

In [2]:
import struct
from typing import NamedTuple, List
from enum import IntEnum

with open('./livelink.udp', 'rb') as udp:
    livelink_bytes = udp.read()

In [ ]:
# ported from https://github.com/EpicGames/UnrealEngine

# FILE: /Users/awkii/Documents/practice/UnrealEngine/Engine/Plugins/Runtime/AR/AppleAR/AppleARKitFaceSupport/Source/AppleARKitFaceSupport/Private/AppleARKitLiveLinkSource.cpp
# READ(): void FAppleARKitLiveLinkRemoteListener::Tick(float DeltaTime)
# WRITE(): void FAppleARKitLiveLinkRemotePublisher::PublishBlendShapes(FName SubjectName, const FQualifiedFrameTime& FrameTime, const FARBlendShapeMap& FaceBlendShapes, FName DeviceId)

In [3]:
class BlendShapes(IntEnum):
    # Left eye blend shapes
    EyeBlinkLeft = 0
    EyeLookDownLeft = 1
    EyeLookInLeft = 2
    EyeLookOutLeft = 3
    EyeLookUpLeft = 4
    EyeSquintLeft = 5
    EyeWideLeft = 6
    # Right eye blend shapes
    EyeBlinkRight = 7
    EyeLookDownRight = 8
    EyeLookInRight = 9
    EyeLookOutRight = 10
    EyeLookUpRight = 11
    EyeSquintRight = 12
    EyeWideRight = 13
    # Jaw blend shapes
    JawForward = 14
    JawLeft = 15
    JawRight = 16
    JawOpen = 17
    # Mouth blend shapes
    MouthClose = 18
    MouthFunnel = 19
    MouthPucker = 20
    MouthLeft = 21
    MouthRight = 22
    MouthSmileLeft = 23
    MouthSmileRight = 24
    MouthFrownLeft = 25
    MouthFrownRight = 26
    MouthDimpleLeft = 27
    MouthDimpleRight = 28
    MouthStretchLeft = 29
    MouthStretchRight = 30
    MouthRollLower = 31
    MouthRollUpper = 32
    MouthShrugLower = 33
    MouthShrugUpper = 34
    MouthPressLeft = 35
    MouthPressRight = 36
    MouthLowerDownLeft = 37
    MouthLowerDownRight = 38
    MouthUpperUpLeft = 39
    MouthUpperUpRight = 40
    # Brow blend shapes
    BrowDownLeft = 41
    BrowDownRight = 42
    BrowInnerUp = 43
    BrowOuterUpLeft = 44
    BrowOuterUpRight = 45
    # Cheek blend shapes
    CheekPuff = 46
    CheekSquintLeft = 47
    CheekSquintRight = 48
    # Nose blend shapes
    NoseSneerLeft = 49
    NoseSneerRight = 50
    TongueOut = 51
    # Treat the head rotation as curves for LiveLink support
    HeadYaw = 52
    HeadPitch = 53
    HeadRoll = 54
    # Treat eye rotation as curves for LiveLink support
    LeftEyeYaw = 55
    LeftEyePitch = 56
    LeftEyeRoll = 57
    RightEyeYaw = 58
    RightEyePitch = 59
    RightEyeRoll = 60

In [4]:
len(BlendShapes)

61

In [5]:
class FrameTime:
    framenum: int
    subframe: int
    fps_numerator: int
    fps_denominator: int

class Livelink(NamedTuple):
    packet_ver: int
    device_id: str
    subject_name: str
    frametime: FrameTime
    blendshape_count: int
    face_blend_shapes_values: List[float] 


class LivelinkBuffer:
    def __init__(self, buffer, offset=0):
        self.buffer = buffer
        self.offset = offset

    def parse(self, unpack_format: str):
        if not unpack_format.startswith('>') and not unpack_format.startswith('<'):
            unpack_format = '>' + unpack_format
        retval = struct.unpack_from(unpack_format, self.buffer, self.offset)
        self.offset += struct.calcsize(unpack_format)
        return retval
    
    def parse_str(self):
        length, *_ = self.parse('i')
        if length > 0:
            retval = self.buffer[self.offset:self.offset+length]
            self.offset += length
            return retval.decode('utf8')
        return ''
    
    def parse_frametime(self):
        return FrameTime(*snip.parse('ifii'))
    
    def parse_livelink(self):
        packet_ver, *_ = self.parse('B')
        if packet_ver != 6:
            raise NotImplementedError("Cannot support packet version: " + str(packet_ver))
        device_id, *_ = self.parse_str()
        subject_name = self.parse_str()
        frametime = self.parse('ifii')
        blendshape_count, *_ = self.parse('B')
        # In Unreal Engine code you have a check condition before
        # blendshape_count is serialized. If the next byte is
        # equal to the packet version, this is the start of a new packet
        if blendshape_count != len(BlendShapes):
            self.offset -= struct.calcsize('B')
            return None
        face_blend_shapes_values = self.parse('f' * blendshape_count)
        return Livelink(
            packet_ver, device_id, subject_name,
            frametime,
            blendshape_count, face_blend_shapes_values
        )


In [9]:
snip = LivelinkBuffer(livelink_bytes)

most_min = 2
most_max = -1
total = 0

# These values should be between 0 to 1.
# Negatives occastionally occur, and should be ignored.
while snip.offset < len(livelink_bytes):
    livelink = snip.parse_livelink()
    if livelink == None:
        continue
    mmin = min(livelink.face_blend_shapes_values)
    mmax = max(livelink.face_blend_shapes_values)
    if mmin < most_min:
        most_min = mmin
    if mmax > most_max:
        most_max = mmax
    total += 1

most_min, most_max, error_count, total

(-0.19074633717536926, 1.0, 0, 560)

In [10]:
livelink

Livelink(packet_ver=6, device_id='1', subject_name='Ahmeds_iPhone', frametime=(1096414, 0.2552375793457031, 60, 1), blendshape_count=61, face_blend_shapes_values=(0.6528505682945251, 0.49640795588493347, 0.11688006669282913, 0.0, 0.0, 0.05742459371685982, 0.0, 0.6556288003921509, 0.49774667620658875, 9.538116864860058e-05, 0.0, 0.0, 0.05742422491312027, 0.0, 0.023357238620519638, 0.0052541689947247505, 0.0, 0.0554061233997345, 0.07145385444164276, 0.09527645260095596, 0.36399880051612854, 0.0, 0.021346818655729294, 0.0, 0.0, 0.07755988091230392, 0.06911591440439224, 0.017455056309700012, 0.01804226078093052, 0.04070117697119713, 0.042244505137205124, 0.05893817916512489, 0.01590118370950222, 0.14774569869041443, 0.11772327125072479, 0.06696517020463943, 0.06838994473218918, 0.027602296322584152, 0.02812124229967594, 0.01904251240193844, 0.021279558539390564, 0.09746091067790985, 0.09731905162334442, 0.07517478615045547, 0.0, 0.0, 0.18365933001041412, 0.040523797273635864, 0.04445596784